Задание 7. Полная проблема собственных значений (метод Якоби).

In [1]:
import numpy as np
from enum import Enum
from math import sqrt
from typing import Optional
from numpy import linalg as LA
from scipy.linalg import hilbert
import pandas as pd

In [2]:
def radius_of_gershgorin_circle(matrix: np.ndarray, i: int) -> float:
    return sum(abs(matrix[i])) - abs(matrix[i, i])


def check_gershgorin_circles(matrix: np.ndarray, eigenvalues: np.ndarray) -> bool:
    size = matrix.shape[0]
    radiuses = [radius_of_gershgorin_circle(matrix, i) for i in range(size)]
    for value in eigenvalues:
        if all(abs(value - matrix[i, i]) > radiuses[i] for i in range(size)):
            return False
    return True

In [3]:
class SelectionStrategy(Enum):
    ABS_MAXIMUM = 1,
    OPTIMAL_SELECTION = 2


def index_of_nondiag_abs_max(matrix: np.ndarray) -> tuple[int, int]:
    matrix = abs(matrix)
    np.fill_diagonal(matrix, -1)
    return np.unravel_index(np.argmax(matrix), matrix.shape)

In [4]:
def create_rotation_matrix(matrix: np.ndarray, index: tuple[int, int]) -> np.ndarray:
    result = np.identity(matrix.shape[0])
    i, j = index
    x, y = -2 * matrix[index], matrix[i, i] - matrix[j, j] 

    if y == 0:
        cos_phi, sin_phi = 1 / np.sqrt(2), 1 / np.sqrt(2)
    else:
        cos_phi = sqrt(1 / 2 * (1 + abs(y) / sqrt(x ** 2 + y ** 2)))
        sin_phi = np.sign(x * y) * abs(x) / (2 * cos_phi * sqrt(x ** 2 + y ** 2))
    
    result[i, i], result[j, j] = cos_phi, cos_phi
    result[i, j], result[j, i] = -sin_phi, sin_phi
    return result


def find_eigenvalues(matrix: np.ndarray, accuracy: float, strategy: SelectionStrategy, limit=10000) -> tuple[np.ndarray | None, int]:
    size = matrix.shape[0]
    radiuses = [radius_of_gershgorin_circle(matrix, i) for i in range(size)]
    if all(r < accuracy for r in radiuses):
        return np.diagonal(matrix), 0

    for step in range(1, limit):
        match strategy:
            case SelectionStrategy.ABS_MAXIMUM: 
                i, j = index_of_nondiag_abs_max(matrix)
            case SelectionStrategy.OPTIMAL_SELECTION:
                i = np.argmax(radiuses)
                row = abs(matrix[i])
                row[i] = -1
                j = np.argmax(row)
        
        rotation_matrix = create_rotation_matrix(matrix, (i, j))
        matrix = rotation_matrix @ matrix @ LA.inv(rotation_matrix)

        for index in [i, j]:
            radiuses[index] = radius_of_gershgorin_circle(matrix, index)

        if all(r < accuracy for r in radiuses):
            return np.diagonal(matrix), step
        
    return None, step

In [5]:
strategies = {
    'максимальный по модулю': SelectionStrategy.ABS_MAXIMUM,
    'оптимальный выбор': SelectionStrategy.OPTIMAL_SELECTION
}

columns = ['точность', 'стратегия', 'количество итераций',
           'ср. погрешность', 'макс. погрешность']

format = {columns[0]: '{:.0e}',
          columns[3]: '{:.6e}',
          columns[4]: '{:.6e}'}

style = [{'selector': 'th', 'props': [('max-width', '100px')]}]


def results_df(matrix: np.ndarray) -> pd.DataFrame:
    exact_values = np.sort(LA.eig(matrix)[0])
    accuracy = [10 ** -p for p in range(2, 11)]
    data = []

    for a in accuracy:
        for s_name, s in strategies.items():
            values, steps = find_eigenvalues(matrix, a, s)
            values = np.sort(values)
            assert check_gershgorin_circles(matrix, values)
            
            errors = [abs(exact_values[i] - values[i]) for i in range(len(values))]
            avg_error = sum(errors) / len(errors)
            max_error = max(errors)
            data.append([a, s_name, steps, avg_error, max_error])

    return pd.DataFrame(data, columns=columns)

### Матрица Гильберта

In [6]:
matrix = hilbert(10)
results_df(matrix).style.format(format).set_table_styles(style).hide()

точность,стратегия,количество итераций,ср. погрешность,макс. погрешность
1e-02,максимальный по модулю,37,3.410909e-04,1.610118e-03
1e-02,оптимальный выбор,38,3.384300e-04,1.620751e-03
1e-03,максимальный по модулю,55,2.296553e-05,9.305773e-05
1e-03,оптимальный выбор,55,2.303134e-05,9.291243e-05
1e-04,максимальный по модулю,70,3.616987e-06,1.754276e-05
1e-04,оптимальный выбор,74,9.596443e-07,4.026606e-06
1e-05,максимальный по модулю,79,5.685204e-07,2.834025e-06
1e-05,оптимальный выбор,82,4.967278e-07,2.454160e-06
1e-06,максимальный по модулю,99,1.950338e-09,9.293967e-09
1e-06,оптимальный выбор,92,2.563747e-09,1.154679e-08


### Трехдиагональная матрица

In [7]:
size = 10
matrix = np.identity(size) + -3 * np.eye(size, k=-1) + -3 * np.eye(size, k=1)
results_df(matrix).style.format(format).set_table_styles(style).hide()

точность,стратегия,количество итераций,ср. погрешность,макс. погрешность
1e-02,максимальный по модулю,96,1.075056e-06,2.160398e-06
1e-02,оптимальный выбор,98,2.857695e-06,6.257568e-06
1e-03,максимальный по модулю,111,3.804082e-08,9.910516e-08
1e-03,оптимальный выбор,113,4.198463e-08,8.041062e-08
1e-04,максимальный по модулю,119,1.461639e-10,3.112435e-10
1e-04,оптимальный выбор,127,1.299372e-10,3.337277e-10
1e-05,максимальный по модулю,128,3.501571e-12,1.041922e-11
1e-05,оптимальный выбор,132,6.858575e-12,2.568967e-11
1e-06,максимальный по модулю,138,2.956801e-14,7.460699e-14
1e-06,оптимальный выбор,142,9.631185e-15,2.575717e-14


### Матрица с диагональным преобладанием

In [8]:
size = 5
matrix = 10 * np.identity(size) + -2 * np.eye(size, k=1) + 2 * np.eye(size, k=2) + 3 * np.eye(size, k=4)
matrix += np.triu(matrix, k=1).T
results_df(matrix).style.format(format).set_table_styles(style).hide()

точность,стратегия,количество итераций,ср. погрешность,макс. погрешность
1e-02,максимальный по модулю,19,1.371358e-05,2.991529e-05
1e-02,оптимальный выбор,20,1.360823e-05,3.401904e-05
1e-03,максимальный по модулю,22,6.052973e-09,1.492072e-08
1e-03,оптимальный выбор,23,2.976416e-08,7.287479e-08
1e-04,максимальный по модулю,24,7.759159e-10,1.728073e-09
1e-04,оптимальный выбор,24,3.088235e-09,6.184996e-09
1e-05,максимальный по модулю,27,3.085532e-13,7.691625e-13
1e-05,оптимальный выбор,27,1.918465e-14,4.085621e-14
1e-06,максимальный по модулю,28,2.842171e-15,4.440892e-15
1e-06,оптимальный выбор,27,1.918465e-14,4.085621e-14
